# MIDI Right Hand Extraction

This notebook demonstrates how to extract only the right hand (melody) from a piano MIDI file using music21.

In [1]:
import music21
from music21 import converter, stream, note, chord
import os

# Set the path to your MIDI file
midi_file_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise.mid"

print(f"Loading MIDI file: {midi_file_path}")
print(f"File exists: {os.path.exists(midi_file_path)}")

Loading MIDI file: /Users/baobach/Projects/splyne/tests/test_data/fur_elise.mid
File exists: True


In [2]:
# Load the MIDI file
score = converter.parse(midi_file_path)

# Display basic information about the score
print("Score information:")
print(f"Number of parts: {len(score.parts)}")
for i, part in enumerate(score.parts):
    print(f"Part {i}: {part.partName if part.partName else 'Unnamed'}")
    print(f"  Number of measures: {len(part.getElementsByClass('Measure'))}")
    print(f"  Instrument: {part.getInstrument()}")
    print()

Score information:
Number of parts: 2
Part 0: Piano
  Number of measures: 126
  Instrument: : 

Part 1: Piano
  Number of measures: 47
  Instrument: : 



In [3]:
def extract_right_hand_melody(score):
    """
    Extract the right hand (melody) from a piano score.
    
    This function uses several heuristics to identify the right hand:
    1. Look for parts with higher average pitch
    2. Consider parts that are more melodic (fewer simultaneous notes)
    3. Use track ordering (typically right hand is first or has higher notes)
    """
    
    if len(score.parts) == 0:
        print("No parts found in the score")
        return None
    
    # Method 1: If there are exactly 2 parts, assume right hand is the one with higher average pitch
    if len(score.parts) == 2:
        print("Found 2 parts - analyzing pitch ranges...")
        
        part1_pitches = []
        part2_pitches = []
        
        # Collect all pitches from both parts
        for element in score.parts[0].flat.notes:
            if isinstance(element, note.Note):
                part1_pitches.append(element.pitch.midi)
            elif isinstance(element, chord.Chord):
                part1_pitches.extend([p.midi for p in element.pitches])
        
        for element in score.parts[1].flat.notes:
            if isinstance(element, note.Note):
                part2_pitches.append(element.pitch.midi)
            elif isinstance(element, chord.Chord):
                part2_pitches.extend([p.midi for p in element.pitches])
        
        if part1_pitches and part2_pitches:
            avg_pitch_1 = sum(part1_pitches) / len(part1_pitches)
            avg_pitch_2 = sum(part2_pitches) / len(part2_pitches)
            
            print(f"Part 0 average pitch: {avg_pitch_1:.1f} (MIDI note number)")
            print(f"Part 1 average pitch: {avg_pitch_2:.1f} (MIDI note number)")
            
            # Right hand typically has higher pitch
            if avg_pitch_1 > avg_pitch_2:
                print("Selecting Part 0 as right hand (higher average pitch)")
                return score.parts[0]
            else:
                print("Selecting Part 1 as right hand (higher average pitch)")
                return score.parts[1]
    
    # Method 2: For single part or multiple parts, look for the most melodic part
    print("Analyzing melodic content...")
    best_part = None
    highest_avg_pitch = -1
    
    for i, part in enumerate(score.parts):
        pitches = []
        for element in part.flat.notes:
            if isinstance(element, note.Note):
                pitches.append(element.pitch.midi)
            elif isinstance(element, chord.Chord):
                # For chords, take the highest note as melody indication
                pitches.append(max([p.midi for p in element.pitches]))
        
        if pitches:
            avg_pitch = sum(pitches) / len(pitches)
            print(f"Part {i} average pitch: {avg_pitch:.1f}")
            
            if avg_pitch > highest_avg_pitch:
                highest_avg_pitch = avg_pitch
                best_part = part
    
    if best_part is not None:
        print(f"Selected part with highest average pitch as right hand")
        return best_part
    
    # Fallback: return the first part
    print("Fallback: returning first part")
    return score.parts[0]

# Extract the right hand
right_hand = extract_right_hand_melody(score)

Found 2 parts - analyzing pitch ranges...
Part 0 average pitch: 72.0 (MIDI note number)
Part 1 average pitch: 52.7 (MIDI note number)
Selecting Part 0 as right hand (higher average pitch)


/Users/baobach/anaconda3/envs/splyne/lib/python3.10/site-packages/music21/stream/base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [4]:
# Create a new score with only the right hand
melody_score = stream.Score()
melody_score.append(right_hand)

# Display some information about the extracted melody
print("Right hand melody extracted!")
print(f"Number of notes/chords: {len(right_hand.flat.notes)}")

# Show first few notes
print("\nFirst 10 elements:")
for i, element in enumerate(right_hand.flat.notes[:10]):
    if isinstance(element, note.Note):
        print(f"  {i+1}. Note: {element.pitch.name} (MIDI: {element.pitch.midi})")
    elif isinstance(element, chord.Chord):
        pitches = [p.name for p in element.pitches]
        print(f"  {i+1}. Chord: {pitches}")
    else:
        print(f"  {i+1}. {type(element).__name__}: {element}")

Right hand melody extracted!
Number of notes/chords: 493

First 10 elements:
  1. Note: E (MIDI: 76)
  2. Note: E- (MIDI: 75)
  3. Note: E (MIDI: 76)
  4. Note: E- (MIDI: 75)
  5. Note: E (MIDI: 76)
  6. Note: B (MIDI: 71)
  7. Note: D (MIDI: 74)
  8. Note: C (MIDI: 72)
  9. Note: A (MIDI: 69)
  10. Note: C (MIDI: 60)


In [5]:
# Optional: Save the right hand melody to a new MIDI file
output_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise_melody.mid"

try:
    melody_score.write('midi', fp=output_path)
    print(f"Right hand melody saved to: {output_path}")
except Exception as e:
    print(f"Error saving MIDI file: {e}")

# Optional: Also save as MusicXML for better readability
try:
    xml_output_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise_melody.xml"
    melody_score.write('musicxml', fp=xml_output_path)
    print(f"Right hand melody also saved as MusicXML: {xml_output_path}")
except Exception as e:
    print(f"Error saving MusicXML file: {e}")

Right hand melody saved to: /Users/baobach/Projects/splyne/tests/test_data/fur_elise_melody.mid
Right hand melody also saved as MusicXML: /Users/baobach/Projects/splyne/tests/test_data/fur_elise_melody.xml
Right hand melody also saved as MusicXML: /Users/baobach/Projects/splyne/tests/test_data/fur_elise_melody.xml


## Converting ABC Notation to MIDI

Now let's convert the ABC notation file (fur_elise_short.abc) to MIDI using music21:

In [6]:
# Path to the ABC file
abc_file_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise_short.abc"

# Check if the file exists
print(f"ABC file path: {abc_file_path}")
print(f"File exists: {os.path.exists(abc_file_path)}")

# Read and display the ABC content
if os.path.exists(abc_file_path):
    with open(abc_file_path, 'r') as f:
        abc_content = f.read()
    print("\nABC file content:")
    print(abc_content)

ABC file path: /Users/baobach/Projects/splyne/tests/test_data/fur_elise_short.abc
File exists: True

ABC file content:
X:1
T:Fur Elise
T:  Für Elise
T:Beethoven
C:Beethoven
%%score {1}
L:1/8
Q:3/8=60
M:3/8
K:C
V:1 treble nm="Piano" snm="Pno."
V:1
 e/^d/ |: e/^d/e/B/=d/c/ | A z/ C/E/A/ | B z/ E/^G/B/ | c z/ E/e/^d/ | e/^d/e/B/=d/c/ | %6
 A z/ C/E/A/ | B z/ D/c/B/ |1 A z e/^d/ :|2 A z/ B/c/d/ |: e3/2 G/f/e/ | d3/2 F/e/d/ | %12



In [7]:
# Convert ABC to music21 score
try:
    # Parse the ABC file using music21
    abc_score = converter.parse(abc_file_path)
    
    print("ABC file successfully parsed!")
    print(f"Number of parts: {len(abc_score.parts)}")
    
    # Display information about the score
    for i, part in enumerate(abc_score.parts):
        print(f"Part {i}: {part.partName if part.partName else 'Unnamed'}")
        print(f"  Number of measures: {len(part.getElementsByClass('Measure'))}")
        print(f"  Number of notes: {len(part.flat.notes)}")
        
        # Show first few notes
        notes_preview = []
        for j, element in enumerate(part.flat.notes[:5]):
            if isinstance(element, note.Note):
                notes_preview.append(f"{element.pitch.name}")
            elif isinstance(element, chord.Chord):
                chord_notes = [p.name for p in element.pitches]
                notes_preview.append(f"[{','.join(chord_notes)}]")
        print(f"  First few notes: {' '.join(notes_preview)}")
        print()
        
except Exception as e:
    print(f"Error parsing ABC file: {e}")
    abc_score = None

ABC file successfully parsed!
Number of parts: 2
Part 0: Unnamed
  Number of measures: 0
  Number of notes: 0
  First few notes: 

Part 1: Unnamed
  Number of measures: 12
  Number of notes: 49
  First few notes: E D# E D# E



In [ ]:
# Save the ABC score as MIDI - Fixed to handle repeats
if abc_score is not None:
    try:
        # Find the part with actual musical content
        active_part = None
        for i, part in enumerate(abc_score.parts):
            note_count = len(part.flat.notes)
            if note_count > 0:
                print(f"Using Part {i} for MIDI export ({note_count} notes)")
                active_part = part
                break
        
        if active_part is None:
            print("❌ No parts with musical content found")
        else:
            # Create a new score with only the active part
            export_score = stream.Score()
            export_score.append(active_part)
            
            # Define output path for the MIDI file
            midi_output_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise_short.mid"
            
            try:
                # Try to expand repeats first
                print("🔄 Attempting to expand repeats...")
                expanded_score = export_score.expandRepeats()
                expanded_score.write('midi', fp=midi_output_path)
                print(f"✅ MIDI file created successfully with expanded repeats: {midi_output_path}")
            except Exception as repeat_error:
                print(f"⚠️  Repeat expansion failed: {repeat_error}")
                print("🔄 Trying without repeat expansion...")
                
                # Try alternative approach: create a simple copy without repeat processing
                simple_score = stream.Score()
                simple_part = stream.Part()
                
                # Copy notes without repeat structure
                for element in active_part.recurse():
                    if isinstance(element, (note.Note, note.Rest, chord.Chord)):
                        simple_part.append(element)
                
                simple_score.append(simple_part)
                simple_score.write('midi', fp=midi_output_path)
                print(f"✅ MIDI file created successfully (simplified): {midi_output_path}")
            
            # Also save as MusicXML for inspection
            try:
                xml_output_path = "/Users/baobach/Projects/splyne/tests/test_data/fur_elise_short.xml"
                export_score.write('musicxml', fp=xml_output_path)
                print(f"✅ MusicXML file created: {xml_output_path}")
            except Exception as xml_error:
                print(f"⚠️  MusicXML export failed: {xml_error}")
            
            # Verify the MIDI file was created
            if os.path.exists(midi_output_path):
                file_size = os.path.getsize(midi_output_path)
                print(f"MIDI file size: {file_size} bytes")
            else:
                print("❌ MIDI file was not created")
            
    except Exception as e:
        print(f"❌ Error creating MIDI file: {e}")
else:
    print("❌ Cannot create MIDI file - ABC parsing failed")

Using Part 1 for MIDI export (49 notes)
❌ Error creating MIDI file: cannot expand Stream: badly formed repeats or repeat expressions


In [ ]:
def abc_to_midi_robust(abc_file_path, midi_output_path=None, verbose=True):
    """
    Convert an ABC notation file to MIDI using music21.
    Handles repeat expressions and finds the correct part automatically.
    
    Args:
        abc_file_path (str): Path to the input ABC file
        midi_output_path (str, optional): Path for the output MIDI file. 
                                        If None, uses the same name with .mid extension
        verbose (bool): Whether to print status messages
    
    Returns:
        tuple: (music21.stream.Score, str) - The parsed score and output path, or (None, None) if failed
    """
    try:
        # Check if input file exists
        if not os.path.exists(abc_file_path):
            if verbose:
                print(f"❌ ABC file not found: {abc_file_path}")
            return None, None
        
        # Parse the ABC file
        if verbose:
            print(f"📖 Parsing ABC file: {abc_file_path}")
        
        score = converter.parse(abc_file_path)
        
        if verbose:
            print(f"✅ Successfully parsed ABC file!")
            print(f"   Total parts found: {len(score.parts)}")
        
        # Find the part with actual musical content
        active_parts = []
        for i, part in enumerate(score.parts):
            note_count = len(part.flat.notes)
            measure_count = len(part.getElementsByClass('Measure'))
            if note_count > 0:
                active_parts.append((i, part, note_count, measure_count))
                if verbose:
                    print(f"   Part {i}: {note_count} notes, {measure_count} measures")
        
        if not active_parts:
            if verbose:
                print("❌ No parts with musical content found")
            return None, None
        
        # Use the part with the most notes
        best_part_info = max(active_parts, key=lambda x: x[2])
        part_idx, active_part, note_count, measure_count = best_part_info
        
        if verbose:
            print(f"🎵 Selected Part {part_idx} for export ({note_count} notes, {measure_count} measures)")
        
        # Create a new score with only the active part
        export_score = stream.Score()
        export_score.append(active_part)
        
        # Generate output path if not provided
        if midi_output_path is None:
            base_name = os.path.splitext(abc_file_path)[0]
            midi_output_path = f"{base_name}.mid"
        
        # Try multiple approaches to handle repeats
        success = False
        
        # Method 1: Try expanding repeats
        if verbose:
            print(f"💾 Attempting MIDI export with repeat expansion...")
        try:
            expanded_score = export_score.expandRepeats()
            expanded_score.write('midi', fp=midi_output_path)
            success = True
            if verbose:
                print(f"✅ MIDI created with expanded repeats!")
        except Exception as e:
            if verbose:
                print(f"⚠️  Repeat expansion failed: {e}")
        
        # Method 2: Try direct export without repeat expansion
        if not success:
            if verbose:
                print(f"💾 Attempting direct MIDI export...")
            try:
                export_score.write('midi', fp=midi_output_path)
                success = True
                if verbose:
                    print(f"✅ MIDI created with direct export!")
            except Exception as e:
                if verbose:
                    print(f"⚠️  Direct export failed: {e}")
        
        # Method 3: Create simplified version without repeat structures
        if not success:
            if verbose:
                print(f"💾 Creating simplified MIDI without repeat structures...")
            try:
                simple_score = stream.Score()
                simple_part = stream.Part()
                
                # Copy only notes, rests, and chords without repeat markers
                for element in active_part.recurse():
                    if isinstance(element, (note.Note, note.Rest, chord.Chord)):
                        # Create a copy to avoid modifying original
                        new_element = element.__deepcopy__()
                        simple_part.append(new_element)
                
                simple_score.append(simple_part)
                simple_score.write('midi', fp=midi_output_path)
                success = True
                if verbose:
                    print(f"✅ MIDI created with simplified approach!")
            except Exception as e:
                if verbose:
                    print(f"❌ Simplified export also failed: {e}")
        
        # Verify file creation
        if success and os.path.exists(midi_output_path):
            file_size = os.path.getsize(midi_output_path)
            if verbose:
                print(f"✅ MIDI file created successfully! ({file_size} bytes)")
            return export_score, midi_output_path
        else:
            if verbose:
                print("❌ MIDI file creation failed")
            return None, None
            
    except Exception as e:
        if verbose:
            print(f"❌ Error converting ABC to MIDI: {e}")
        return None, None

# Test the robust function
print("=== Testing Robust ABC to MIDI Conversion ===")
result_score, output_file = abc_to_midi_robust(
    abc_file_path="/Users/baobach/Projects/splyne/tests/test_data/fur_elise_short.abc",
    midi_output_path="/Users/baobach/Projects/splyne/tests/test_data/fur_elise_short_robust.mid"
)

if result_score and output_file:
    print(f"\n🎉 Success! MIDI file created at: {output_file}")
else:
    print("\n❌ Failed to create MIDI file")